In [18]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# Load your Excel file (replace 'Pushing.xlsx' with the actual path if needed)
#file_path = 'Pushing.xlsx'
#xls = pd.ExcelFile(file_path)

#file_path = "C:\\single-dual-device-comparison\\Pushing.xlsx"
#pushing1 = pd.read_excel(file_path, sheet_name='Pushing1')
#pushing2 = pd.read_excel(file_path, sheet_name='Pushing2')

#file_path = "C:\\single-dual-device-comparison\\Rolling.xlsx"
#pushing1 = pd.read_excel(file_path, sheet_name='Rolling1')
#pushing2 = pd.read_excel(file_path, sheet_name='Rolling2')

file_path = "C:\\single-dual-device-comparison\\PushingEdges.xlsx"
pushing1 = pd.read_excel(file_path, sheet_name='PushingEdges1')
pushing2 = pd.read_excel(file_path, sheet_name='PushingEdges2')
# Load the two sheets
#pushing1 = pd.read_excel(xls, 'Pushing1')
#pushing2 = pd.read_excel(xls, 'Pushing2')

# Binary mapping: 'Different' = 1, others = 0
category_to_binary = {
    'Different': 1,
    'VS': 0,
    'Indifferent': 0
}

# Convert categorical data to binary (skip 'RefreshRate')
pushing1_binary = pushing1.copy()
pushing2_binary = pushing2.copy()
for col in pushing1.columns:
    if col != 'RefreshRate':
        pushing1_binary[col] = pushing1[col].map(category_to_binary)
        pushing2_binary[col] = pushing2[col].map(category_to_binary)

# Function to perform McNemar's test for each row
def perform_mcnemar_test(sheet1, sheet2):
    results = []
    for index in range(len(sheet1)):
        # Initialize counts for the 2x2 contingency table
        a = 0  # Both 0
        b = 0  # Sheet1: 0, Sheet2: 1
        c = 0  # Sheet1: 1, Sheet2: 0
        d = 0  # Both 1
        
        # Iterate over user columns (skip 'RefreshRate')
        for col in sheet1.columns:
            if col == 'RefreshRate':
                continue
            val1 = sheet1.at[index, col]
            val2 = sheet2.at[index, col]
            if val1 == 0 and val2 == 0:
                a += 1
            elif val1 == 0 and val2 == 1:
                b += 1
            elif val1 == 1 and val2 == 0:
                c += 1
            elif val1 == 1 and val2 == 1:
                d += 1
        
        # Create the contingency table for McNemar's test (focus on b and c)
        table = [[a + d, b], [c, 0]]
        
        # Perform the test (exact=True for small samples)
        result = mcnemar(table, exact=True)
        
        # Store results
        results.append({
            'row': index,
            'RefreshRate': pushing1.at[index, 'RefreshRate'],  # From Pushing1 (assuming aligned)
            'statistic': result.statistic,
            'pvalue': result.pvalue,
            'significant': result.pvalue < 0.05  # Common significance threshold
        })
    return pd.DataFrame(results)

# Run the function and display results
results_df = perform_mcnemar_test(pushing1_binary, pushing2_binary)
print(results_df)

# Optional: Save results to CSV
# results_df.to_csv('mcnemar_results.csv', index=False)


    row  RefreshRate  statistic  pvalue  significant
0     0           50        0.0   1.000        False
1     1          100        0.0   1.000        False
2     2          200        0.0   1.000        False
3     3          300        0.0   1.000        False
4     4          400        0.0   1.000        False
5     5          500        0.0   1.000        False
6     6          600        1.0   1.000        False
7     7          700        1.0   1.000        False
8     8          800        0.0   0.125        False
9     9          900        0.0   0.250        False
10   10          950        0.0   1.000        False
11   11         1000        0.0   1.000        False
